In [1]:
# Library imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader as TorchDataLoader
import pandas as pd

# Our imports
from DL_vs_HateSpeech.loading_data.dataloader import DataLoader
from DL_vs_HateSpeech.training.training import collate_fn
from DL_vs_HateSpeech.utils import check_frozen_params
from DL_vs_HateSpeech.models.utils import load_model_from_path
from DL_vs_HateSpeech.evaluation.evaluate import evaluate


# Some constants
DATA_SUBSET = "us_pol"
BATCH_SIZE = 1


# Load Data
train_dataset = DataLoader(type="train", subset=DATA_SUBSET)
test_dataset = DataLoader(type="test", subset=DATA_SUBSET)
train_loader = TorchDataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = TorchDataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)


c:\Users\mattb\anaconda3\envs\DLHS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Best Models

In [2]:
# Dataset to store the model accuracies and F1 scores
df = pd.DataFrame(columns=["model", "accuracy", "f1_score_0", "f1_score_1"])

## Best model using CLIP of type 16

### With augmentation

In [ ]:
path = "./ModelV2_single_class_clip_16_testing_False"
model_v2_16 = load_model_from_path(path, file_name="model_epoch_2_ac_0.647887323943662.pth")
model_v2_16.eval()

# Check how many parameters are frozen
check_frozen_params(model_v2_16)

Trainable params: 4473089, Frozen params: 149620737


In [3]:
avg_loss_test, accuracy_test, f1_test = evaluate(model_v2_16, test_loader, nn.BCEWithLogitsLoss(), device="cpu")

Evaluating: 100%|██████████| 355/355 [01:44<00:00,  3.40it/s]


In [4]:
print(f"Average Loss: {avg_loss_test:.4f}")
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"F1 Score label 0: {f1_test[0] * 100:.2f}%")
print(f"F1 Score label 1: {f1_test[1] * 100:.2f}%")

Average Loss: 0.6341
Accuracy: 65.07%
F1 Score label 0: 68.69%
F1 Score label 1: 60.51%


## Best model using CLIP of type 32

In [5]:
path = ".\models\model_checkpoints\ModelV2_single_class_clip_32_testing_False"
model_v2_32 = load_model_from_path(path, file_name="model_epoch_28_ac_0.6225352112676056.pth")
model_v2_32.eval()

# Check how many parameters are frozen
check_frozen_params(model_v2_32)

Trainable params: 4473089, Frozen params: 151277313


In [6]:
avg_loss_test, accuracy_test, f1_test = evaluate(model_v2_16, test_loader, nn.BCEWithLogitsLoss(), device="cpu")

Evaluating: 100%|██████████| 355/355 [01:41<00:00,  3.51it/s]


In [7]:
print(f"Average Loss: {avg_loss_test:.4f}")
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"F1 Score label 0: {f1_test[0] * 100:.2f}%")
print(f"F1 Score label 1: {f1_test[1] * 100:.2f}%")

Average Loss: 0.6348
Accuracy: 65.63%
F1 Score label 0: 69.35%
F1 Score label 1: 60.90%
